# Import Libraries

In [103]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import urllib3

# Define Functions

In [104]:
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
            else:
                sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)

def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll("span", attrs={"class": "location"})
    for span in spans:
        locations.append(span.text)
    return(locations)

def extract_salary_from_result(soup): 
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find("nobr").text)
        except:
            try:
                div_two = div.find(name="div", attrs={"class":"salarySnippet salarySnippetDemphasize"})
                div_three = div_two.find(name="span", attrs={"class":"salary no-wrap"})
                salaries.append(div_three.text.strip())
            except:
                salaries.append("Nothing_found")
    return(salaries)

def extract_links_from_result(soup): 
    links = []
    lnk_final = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="div", attrs={"class":"title"}):
            for b in a.find_all(name="a", attrs={"class":"jobtitle turnstileLink", "data-tn-element":"jobTitle"}):
                links.append(b["href"])    
    return(links)

def extract_skills_from_result(link):
    try:
        page = requests.get(link,timeout=1)
        soup = BeautifulSoup(page.text, "html.parser")
        result = soup.find("div", {"id":"jobDescriptionText", "class":"jobsearch-jobDescriptionText"})
        return result.text
    except:
        return("")

# Define Job Position and scrape related data

In [105]:
loc=[]
sal=[]
ttl=[]
cmp=[]
lnk=[]
job_desc=[]
job_role="Statistical Data Analyst"
location="Ontario"

for i in range(0,500,10):
    URL = ("https://www.indeed.ca/jobs?q="+urllib.parse.quote_plus(job_role)+"&l="+urllib.parse.quote_plus(location)+"&fromage=15&start="+str(i))
    page = requests.get(URL)
    soup = BeautifulSoup(page.text, "html.parser")
#     print(soup.prettify())
    for x in extract_job_title_from_result(soup):
        ttl.append(x)
    for x in extract_salary_from_result(soup):
        sal.append(x)
    for x in extract_company_from_result(soup):
        cmp.append(x)
    for x in extract_location_from_result(soup):
        loc.append(x)
    for x in extract_links_from_result(soup):
        lnk.append(x)

In [106]:
from IPython.display import clear_output
print(len(lnk))
for x in range(0,len(lnk)):
    job_desc.append(extract_skills_from_result("https://ca.indeed.com"+lnk[x]))
#     print(x)

1285


# Filter Skills from Job Description

In [107]:
skills=["Python","R","Tableau","AWS","Azure","C#","Java","PowerBI","Microsoft", "SQL","MySQL","PostgreSQL","SSRS","SSIS","Office", "Powerpoint","Excel","Word","Hadoop","Hive","Splunk","Pig","Spark","Scala","Cloudera","C++","Linux", "SAS","Access","NoSQL", "JavaScript", "MongoDB", "Matlab"]
seperator = ', '
final_skills=[]
final_tab=[]
for j in job_desc:
    for i in skills:
        if i.lower() in j.lower():
            final_skills.append(i)
    final_tab.append(seperator.join(final_skills))
    final_skills.clear()

In [108]:
len(final_tab)

1285

# Create Excel File from the scraped Data

In [109]:
columns = {}

columns["JobTitle"] = ttl 
columns["Salary"] = sal
columns["CompanyName"] = cmp
columns["Location"] = loc
columns["Skills"] = final_tab

data = list(zip(columns["JobTitle"],columns["Salary"],columns["CompanyName"],columns["Location"],columns["Skills"]))

df = pd.DataFrame(data = data)

df.to_csv("D://jobs_"+job_role+".csv", index=False, header=False)

In [110]:
print(len(ttl))
print(len(sal))
print(len(cmp))
print(len(loc))
print(len(final_tab))

1285
1285
1428
988
1285
